In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tqdm
!pip install cv2

In [ ]:
import numpy as np 
import cv2 # OPENCV keep it as it is to avoid confusion among versions
import os
from tqdm import tqdm # Helps show progress on loops

In [ ]:
img = cv2.imread('/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/0.jpg')

import matplotlib.pyplot as plt
plt.imshow(img)

In [ ]:
class CatsDogs():
    catpath = '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat'
    dogpath = '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog'
    imgSize = 50
    nCats = 0
    nDogs = 0
    training_data = []
    
    def getData(self):
        # Get all cat images
        for i in tqdm(os.listdir(path = self.catpath)):
            filepath = os.path.join(self.catpath,i)
            try:
                img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (self.imgSize, self.imgSize))
                label = np.eye(2)[0] # Assume cat has OHE vector [1, 0]
                self.training_data.append([np.array(img),label])
                self.nCats += 1
            except:
                pass
        # Get all dog Images
        for i in tqdm(os.listdir(path = self.dogpath)):
            filepath = os.path.join(self.dogpath,i)
            try:
                img = cv2.imread(filepath,cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (self.imgSize, self.imgSize))
                label = np.eye(2)[1] # Assume dog has OHE vector [0, 1]
                self.training_data.append([np.array(img),label])
                self.nDogs += 1
            except Exception as e:
                pass
        print("hello")
        #print(self.training_data)
        np.random.shuffle(self.training_data)
        np.save("train_data.npy", np.array(self.training_data, dtype=object))
        return self.training_data
obj = CatsDogs()
d = obj.getData()

In [ ]:
print(f"No of cats : {obj.nCats} and No of dogs : {obj.nDogs} ")

In [ ]:
training_data = np.load("train_data.npy", allow_pickle = True)
training_data.shape

In [ ]:
print(training_data[0])

In [ ]:
training_data[0][0]

In [ ]:
plt.imshow(training_data[0][0])

In [ ]:
!pip install torch-vision

In [ ]:
print(2)

In [ ]:
#Torch libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels =32, kernel_size =5)
        # if input image is grayscale/BW then in_channels =1, 
        # else if RBG then in_channels = 3
        # The out_channels is not single filter, In a single layer it is the no of filters 
        # applied on same image, again and again, in all window positions.
        
        # At each position of the input image, a single filter (or kernel) slides over the receptive field (a window of the input) 
        # and performs a convolution operation.
        # The result of this convution operation at each position produces a single value in the output feature map.
        # This process is repeated for each filter in the layer.
        
        # Multiple Filters:
        # Each convolutional layer consists of multiple filters, where the number of filters is specified by 
        # the out_channels parameter.
        # Each filter slides over the entire input image independently.
        # Each filter learns to detect different features or patterns from the input image, such as edges, textures, or shapes.
        # The output of each filter applied to the input image produces a separate feature map.
        
        # kernel_size is the windows size of the kernel e.g. (5x5)
        # Note that stride is also a parameter, indicating by how many pixels should window be shifted to next position 
        # after convultion at each position
        
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5)
        
        # Note that in the 2nd layer, the in_channels is actually (no of filters * no of channels) used in previous layer.
        # Since no of channels in prev layer is 1, hence in_channels in conv2 is simply = no filters in prev layer.
        # Now we can again define, the number out_channels , no of filters in 2nd layer;
        
        # But first, to get the output of 1st layer, we need to define maxpool and activation fn, whose output is the 2nd layer input tensor
        # We can apply below, or use these functions from F in the fly.
        # self.relu = nn.ReLu()
        # self.maxpool = F.max_pool2d()
        # Note that Applying activation function to the output of a max pooling operation does not change the spatial dimensions of the feature maps.
        
        # Next we need to define the Fully Connected layers, since we want a vector or single value for classification, 
        
        self.fc1 = nn.Linear( 12*12 * 64, 256)
        
        # Note that to define fc after Conv, we need to know the shape of output of final max_pool. 
        # In keras we simply use flatten() func, but in Pytorch manual calculation must be done
        
        # For our problem, image h,w = 50,50 and kernel of conv = 5x5.
        # Assume maxpool we define as max_pool2d(input, kernel_size = 2)
        # Typically we define kernel for maxpool as 2x2 or 3x3. Assume we define 2x2
        # But another parameter of maxpool is stride,if you don't specify the stride parameter when calling, it will default to the kernel_size. 
        # Hence, maxpool stride = 2x2. 
        
        #So output of 1st maxpool is 25x25. 
        
        # similarly when we apply 2nd conv which again gives 25x25 output which will be passed to 2nd maxpool,
        # The 2nd maxpool gives 12x12 (it ignores last column of 25 columns).
        
        # Hence the 12*12 * 64 is the size of total output of 2nd maxpool considering all filter channels also.
        # Now simply define other FC layers.
         
        self.fc2 = nn.Linear(256, 2) #since 2 output class
        
    def forward(self, x):
        x = F.relu(conv1(x))
        x = F.max_pool2d(x, kernel_size = 2)
        x = F.relu(conv2(x))
        x = F.max_pool2d(x, kernel_size = 2)
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [ ]:
model = MyCNN()
print(model)

In [ ]:
# We can find the shape of input to be passed to FC layers by passing random input values to initial layers
# and getting the shape of their output, which is desired shape

# This process is commonly referred to as "shape inference" or "forward pass inference"

#SO lets make a TestCNN, for this 

# FORWARD PASS INFERENCE

class TestCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5)
    
    def forward(self, x):
        x = F.relu( self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2)
        x = F.relu( self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2)
        return x
testM = TestCNN()
print(testM)

In [ ]:
# Now Lets make a dummy input x, Remember input to conv2d is of form
# (Batchsize ie no images in a batch, no of channels, height of image, width of image)
# Hence lets define single image batch with only 1 image

x = torch.randn(1,1,50,50)

# Note that same can be achieved with higher batch size, especially during actuall implementation
# using view(-1,other dimesions) command
 
# x = torch.randn(50,50).view(-1,1,50,50) 
#In this case batch size is infered from the shape of input sent and other dimensions

output = testM(x)

print(output.shape)

In [ ]:
# So our earlier Analysis of the architeture is wrong. 

The architecture of the CNN is as follows:

The input size is 50×50 pixels.
The first convolutional layer (conv1) has 32 output channels and a kernel size of 
5×5.

The second convolutional layer (conv2) has 64 output channels and a kernel size of 
5×5.

Between each convolutional layer, there is a max pooling operation with a kernel size of 
2×2.

IMP : Note that Output of conv2d layer is not same size as size of input img. 

 Formula with zero padding  : Output size = ⌊(Input size−Kernel size) / Stride⌋ +1
                                            ⌊( 50 − 5) / 1⌋ +1 = 46

This can also be logically understood since, as we slider the kernel, 
for the last 4 columns, the window doesnt move. Hence output of conv1 is 46x46
 

In [ ]:
# Next few layers analysis : 

After the first max pooling operation:

Output size: 23×23 (since the default stride is equal to the kernel size)

After the second convolutional layer (conv2):

Output size: 19×19 (since (23-5)/1 + 1 = 19, or since (19,20,21,22,23) is the last window. We cant shift further)

After the second max pooling operation:

Output size: 9×9 (since we ignore last single col)

In [ ]:
# Anyway Lets Move on. and Define correct CNN architecure.
class MyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels =32, kernel_size =5)
        
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5)
        
        self.fc1 = nn.Linear( 9 * 9 * 64, 256)
        
        self.fc2 = nn.Linear(256, 2) #since 2 output class
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2)
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim = 1) # Last layer. So act = softmax, since binary classify dim =1
        return x

In [ ]:
model = MyCNN()
print(model)

In [ ]:
#Get Input featureset X and target y
X = []
y = []
for i in training_data:
    X.append(i[0])
    y.append(i[1])
#X = np.array(X)
#y = np.array(y)
X = torch.tensor(np.array(X))
y = torch.tensor(np.array(y))
X = X / 255.0

In [ ]:
X.dtype

In [ ]:
len(X)

In [ ]:
X[0]

In [ ]:
X[0].view(1,50,50)

In [ ]:
X.shape

In [ ]:
# Train Test Split

testSplit = 0.1
testSize = int(len(X)*0.1)
trainSize = len(X) - testSize

Xtrain = X[:trainSize]
Xtest = X[trainSize:]

ytrain = y[:trainSize]
ytest = y[trainSize:]

In [ ]:
Xtrain[0:32].view(-1,1,50,50).shape # take watver is the batch_size but make sure channel is kept as 1, rest H&W

In [ ]:
# See the sample output of model

model(X[0].view(1,1,50,50))

In [ ]:
y[0]

**Training**

In [ ]:
epochs = 20 
lr = 0.01

batch_size = 32

opt = torch.optim.Adam( model.parameters(), lr = 0.00)
mse = nn.CrossEntropyLoss()

val_loss_min = np.inf
early_stop_cnt = 0

for ep in range(epochs):
    for i in tqdm(range(0,len(Xtrain),batch_size)): # Go from 0 to entire length leaving gap of batch_size
        Xtrain_batched = Xtrain[i:i+batch_size].view(-1,1,50,50)
        ytrain_batched = ytrain[i:i+batch_size]
        #print(Xtrain_batched.shape)
        
        # Xtrain_batched = Xtrain_batched.to(torch.float32)
        
        opt.zero_grad()
        
        model.train()
        
        ypred = model(Xtrain_batched)
        
        loss = mse(ypred, ytrain_batched)
        
        loss.backward()
        
        opt.step()
        
    with torch.no_grad():
        model.eval()
        
        Xtest = Xtest.view(-1,1,50,50)
        
        output_val = model(Xtest)
        
        val_loss = mse(output_val, ytest)
        
        print(f" epoch : {ep}, training_loss : {train_loss}")
        
        if val_loss < val_loss_min:
            print(f"val Loss decreased. {val_loss_min} -> {val_loss} ")
            val_loss_min = val_loss
            early_stop_cnt = 0
        else:
            print('val loss same')
            early_stop_cnt +=1
        print(f" Best val loss : {val_loss_min}")
        
        if early_stop_cnt == 10:
            print(" Early Stopping ); ")
            break
        


In [ ]:
test_index = 100

plt.imshow(X[test_index])

In [ ]:
out = model(X[test_index].view(1,1,50,50))
print(y[test_index])
print((out))